# Backward Procedure

El procedimiento de retroceso o bacward procedure es similar al procedimiento de avance, pero la cadena se recorre en sentido inverso, es decir, de derecha a izquierda. Aquí presentamos una implementación simplie de este procedimiento utilizando las propiedades de operaciones entre arreglos en numpy.

In [1]:
from __future__ import division
import numpy as np

En primer lugar, definimos nuestro modelo oculto de Markov, que cuenta con la matriz de transiciones $A$, la matriz de probabilidades de emisiones $B$, el vector de iniciales $\Pi$ y los alfabetos de observaciones $O$ y de emisiones $S$. En este caso, indexamos los alfabetos a partir de estructuras de diccionario.

In [2]:
A = np.array([[1/6,1/6,1/5],[4/6,1/6,1/5],[1/6,3/6,1/5]])
B = np.array([[4/7,1/7,1/6],[1/7,2/7,1/6],[1/7,2/7,1/6],[1/7,2/7,3/6]])
P = np.array([4/6,1/6,1/6])

O = {'la': 0, 'nina': 1, 'garza':2, 'pasa':3}
S = {'DA':0, 'N': 1,'V':2}

El siguiente paso es obtener una cadena de observaciones, $o \in O^*$, y la separamos en sus elementos constituyentes (que en este caso son palabras) para obtener una lista, con la que trabajaremos.

In [3]:
obs = 'la nina la pasa'
obs = obs.split()
print obs

['la', 'nina', 'la', 'pasa']


Ahora es cuando iniciamos con el procedimiento de retroceso. El paso de inducción consiste en determinar las variables $\beta_i(T+1) = 1$. En este caso, utilizamos un vector, que podemos denotar como $\beta_\cdot(T+1)$, el cual contendrá en un primer momento sólo unos.

In [4]:
b = np.ones(len(P))

En el paso inductivo tenemos que obtener las variables:

$$\beta_j(t) = \sum_{i=1^N} p(o_t|s_j)p(s_i|s_j)\beta_i(t+1)$$

Ya que estamos trabajando con arreglos de numpy, podemos ver que la suma y el producto sobre los $i$-ésimos elementos determinan un producto punto. Así, $B_{t,\cdot}$ representa el vector columna correspondiente al elemento de la cadena de observaciones en el estado $t$, $A_{\cdot, j}$ a la $j$-ésima columna de la matriz de transiciones $A$ y $\beta_\cdot(t+1)$ al vector de las variables $\beta$ en el estado subsecuente. Así, podemos expresar la ecuación anterior como:

$$\beta_j(t) = (\beta_\cdot(t+1) \otimes B_{t,\cdot}) \cdot A_{\cdot, j}$$

Donde $\cdot$ es el producto punto, y $\otimes$ el producto de Hadarmard. En términos generales, podemos actualizar todas las variables $\beta$ expresándolo como un vector $\beta_\cdot(t)$ y de esta forma, tenemos que realizar la operación:

$$\beta_\cdot(t) = (\beta_\cdot(t+1) \otimes B_{t,\cdot}) \cdot A$$

Donde el producto punto se hace con toda la matriz de transiciones $A$.

In [5]:
t = len(obs)
for i in range(t):
    b = np.dot( b*B[O[obs[t-i-1]]], A )

Finalmente, el paso de terminación consiste en obtener la probabilidad de la cadena de la siguiente forma:

$$p(o_1,..,o_T) = \sum_{j=1}^N \pi_j \beta_j(1)$$

Donde $\pi_j$ es la $j$-ésima probabilidad inicial. De igual forma, podemos aprovechar la estructura de los arreglos y expresar la suma de los productos por medio de un producto punto con el vector de probabilidades inicailes $\Pi$ y el vector de variables $\beta_\cdot(1)$. Esto es:

$$p(o_1,..,o_T) = \Pi \cdot \beta_\cdot(1)$$


In [6]:
print np.dot(P,b)

0.00207835958268


Por último, podemos comparar nuestros resultados con la variable de avance. Está se resume de la siguiente forma:

In [7]:
a = P
for t in range(len(obs)):
    a = np.dot( A.T, B[O[obs[t]]]*a )
                  
print a.sum(0)

0.00244700647707


Como se puede observar, los resultados obtenidos por ambos métodos son muy similares, lo que muestra que ambos métodos dan una probabilidad cercana a la probabilidad que buscamos sobre una cadena de observaciones.